In [748]:
import pandas as pd

In [749]:
my_prediction = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\predictions_vs_real_with_variant_all_genes_updated_221123.csv")
# change column name from "reality" to "pathogenicity"
my_prediction.rename(columns={"reality": "pathogenicity"}, inplace=True)
my_prediction

,variant,prediction,pathogenicity,gene
0,M1R,0,0,COL2A1
1,I2A,0,0,COL2A1
2,I2L,0,0,COL2A1
3,I2P,0,0,COL2A1
4,I2T,0,0,COL2A1
...,...,...,...,...
4799,Y217F,0,0,GJB2
4800,G220A,0,0,GJB2
4801,G220K,0,0,GJB2
4802,K221R,0,0,GJB2


In [750]:
for gene in my_prediction["gene"].unique():
    print(f"Gene: {gene}")
    print(my_prediction[my_prediction["gene"] == gene]["pathogenicity"].value_counts())

Gene: COL2A1
0    665
1    286
Name: pathogenicity, dtype: int64
Gene: COL4A3
0    264
1    127
Name: pathogenicity, dtype: int64
Gene: COL4A5
1    407
0    227
Name: pathogenicity, dtype: int64
Gene: WFS1
1    172
0    138
Name: pathogenicity, dtype: int64
Gene: SLC26A4
1    265
0     85
Name: pathogenicity, dtype: int64
Gene: MYO7A
0    1192
1     210
Name: pathogenicity, dtype: int64
Gene: FGFR1
0    356
1    133
Name: pathogenicity, dtype: int64
Gene: GJB2
1    204
0     73
Name: pathogenicity, dtype: int64


In [751]:
tools = ['VEST', 'alphamissense', 'REVEL', 'EVE', 'MutPred']

In [752]:
# Create merge, which is the my_prediction dataframe with the predictions of the other tools
merged = my_prediction
for tool in tools:
    tool_predictions = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\{tool}_on_dvd_data_predictions_LOPO.csv")
    # Merge the two files
    merged = pd.merge(merged, tool_predictions, on=["gene", "variant"])
    # remove duplicate rows
    merged = merged.drop_duplicates(subset=["gene", "variant"])
# change column name of pathogenicity_x to pathogenicity
merged = merged.rename(columns={"prediction_x": "prediction", "am_pathogenicity": "am_score", "am_class": "alphamissense_pathogenicity"})

C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3550122812.py:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged = pd.merge(merged, tool_predictions, on=["gene", "variant"])


In [753]:
# Print how many variants there are from each gene

merged["gene"].value_counts()
# my_prediction["gene"].value_counts()
# print how many variants there are from each gene and each pathogenicity
for gene in merged["gene"].unique():
    print(f"Gene: {gene}")
    print(merged[merged["gene"] == gene]["pathogenicity"].value_counts())

Gene: COL2A1
1    254
0    181
Name: pathogenicity, dtype: int64
Gene: COL4A3
0    129
1     92
Name: pathogenicity, dtype: int64
Gene: COL4A5
1    359
0     99
Name: pathogenicity, dtype: int64
Gene: MYO7A
0    381
1    153
Name: pathogenicity, dtype: int64
Gene: FGFR1
1    91
0    75
Name: pathogenicity, dtype: int64


In [754]:
# Print all rows in merged that aren't in my_prediction, according to the variant
merged[merged["variant"].isin(my_prediction["variant"])]

,variant,prediction,pathogenicity,gene,Unnamed: 0_x,VEST_score,VEST_pathogenicity,Unnamed: 0_y,prediction_y,reality_x,...,revel_score,REVEL_pathogenicity,Unnamed: 0_y,prediction,reality_y,EVE_scores_ASM,EVE_pathogenicity,Unnamed: 0,MutPred_score,MutPred_pathogenicity
0,A128V,0,0,COL2A1,162,0.342,0,201,0,0,...,0.215,0,0,0,0,0.082091,0,123,0.259,0
1,D139E,0,0,COL2A1,164,0.150,0,203,0,0,...,0.163,0,2,0,0,0.108896,0,124,0.237,0
2,E142D,0,0,COL2A1,167,0.061,0,206,0,0,...,0.289,0,5,0,0,0.058798,0,127,0.252,0
3,A145P,0,0,COL2A1,168,0.229,0,207,0,0,...,0.272,0,6,0,0,0.033691,0,128,0.238,0
4,A145T,0,0,COL2A1,169,0.314,0,208,0,0,...,0.218,0,7,0,0,0.072494,0,129,0.104,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,R646W,1,1,FGFR1,3494,0.974,1,4234,1,1,...,0.954,1,3938,1,1,0.919227,1,3556,0.929,1
1824,T695I,1,1,FGFR1,3495,0.984,1,4235,1,1,...,0.927,1,3939,1,1,0.894803,1,3557,0.906,1
1825,L712P,1,1,FGFR1,3497,0.975,1,4237,1,1,...,0.959,1,3941,1,1,0.837127,1,3558,0.927,1
1826,E692G,1,1,FGFR1,3498,0.938,1,4238,1,1,...,0.986,1,3942,1,1,0.923482,1,3559,0.943,1


In [755]:

### Notice that VEST4 doesn't have data of WFS1, and REVEL doesn't have data for SLC26A4.

In [756]:
## Calculate the MCC for the predictions, according to the real pathogenicity (0 or 1)
from sklearn.metrics import matthews_corrcoef

In [757]:
# separate the merged dataframe according to gene
genes = merged["gene"].unique()
print(f"Number of genes: {len(genes)}")

Number of genes: 5


In [758]:
# Calculate MCC for each gene specific predictor for mutpred

# Build dictionary with gene names as keys.
mccs = {gene: 0 for gene in genes}

for tool in tools:
    for gene in genes:
        gene_df = merged[merged["gene"] == gene]
        # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
        # Create binary predictions based on the 0.5 threshold
        
        gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
        
        # Get TP, TN, FP, FN
        TP = len(gene_df[(gene_df['pathogenicity'] == 1) & (gene_df[f"{tool}_pathogenicity"] == 1)])
        TN = len(gene_df[(gene_df['pathogenicity'] == 0) & (gene_df[f"{tool}_pathogenicity"] == 0)])
        FP = len(gene_df[(gene_df['pathogenicity'] == 0) & (gene_df[f"{tool}_pathogenicity"] == 1)])
        FN = len(gene_df[(gene_df['pathogenicity'] == 1) & (gene_df[f"{tool}_pathogenicity"] == 0)])
        print(f"{tool} --{gene} TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
        
        # Calculate MCC
        mcc = matthews_corrcoef(gene_df['pathogenicity'], gene_df[f"{tool}_pathogenicity"])
        
        # Get gene name for the use for the dictionary
        gene = gene_df['gene'].unique()[0]
        
        # Append mcc to dictionary
        mccs[gene] = mcc
    
    print(f"MCCs of {tool} predictions for each gene:")
    for gene in mccs:
        print(f"{gene}: {mccs[gene]}")
        
# get MCC for my predictions
mccs = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
    # Create binary predictions based on the 0.5 threshold
    
    gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
    gene_df['prediction'] = gene_df['prediction'].astype(int)
    
    # Get TP, TN, FP, FN
    TP = len(gene_df[(gene_df['pathogenicity'] == 1) & (gene_df['prediction'] == 1)])
    TN = len(gene_df[(gene_df['pathogenicity'] == 0) & (gene_df['prediction'] == 0)])
    FP = len(gene_df[(gene_df['pathogenicity'] == 0) & (gene_df['prediction'] == 1)])
    FN = len(gene_df[(gene_df['pathogenicity'] == 1) & (gene_df['prediction'] == 0)])
    
    print(f"my tool: {gene} TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
    
    # Calculate MCC
    mcc = matthews_corrcoef(gene_df['pathogenicity'], gene_df['prediction'])
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    
    # Append mcc to dictionary
    mccs[gene] = mcc

print(f"MCCs of predictions for each gene:")
for gene in mccs:
    print(f"{gene}: {mccs[gene]}")


C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

VEST --COL2A1 TP: 253, TN: 173, FP: 8, FN: 1
VEST --COL4A3 TP: 91, TN: 123, FP: 6, FN: 1
VEST --COL4A5 TP: 359, TN: 99, FP: 0, FN: 0
VEST --MYO7A TP: 152, TN: 330, FP: 51, FN: 1
VEST --FGFR1 TP: 90, TN: 64, FP: 11, FN: 1
MCCs of VEST predictions for each gene:
COL2A1: 0.9577148337368127
COL4A3: 0.9363283208111267
COL4A5: 1.0
MYO7A: 0.8006551781427285
FGFR1: 0.8588624500892263
alphamissense --COL2A1 TP: 253, TN: 171, FP: 10, FN: 1
alphamissense --COL4A3 TP: 88, TN: 125, FP: 4, FN: 4
alphamissense --COL4A5 TP: 355, TN: 99, FP: 0, FN: 4
alphamissense --MYO7A TP: 151, TN: 329, FP: 52, FN: 2
alphamissense --FGFR1 TP: 89, TN: 64, FP: 11, FN: 2
MCCs of alphamissense predictions for each gene:
COL2A1: 0.9484577548319691
COL4A3: 0.9255139871924503
COL4A5: 0.9749131698112075
MYO7A: 0.7921227814649218
FGFR1: 0.8454073822209278
REVEL --COL2A1 TP: 254, TN: 166, FP: 15, FN: 0
REVEL --COL4A3 TP: 92, TN: 121, FP: 8, FN: 0
REVEL --COL4A5 TP: 359, TN: 97, FP: 2, FN: 0
REVEL --MYO7A TP: 152, TN: 328, FP:

C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

EVE --FGFR1 TP: 79, TN: 67, FP: 8, FN: 12
MCCs of EVE predictions for each gene:
COL2A1: 0.9338297967906009
COL4A3: 0.927626067077041
COL4A5: 0.9749131698112075
MYO7A: 0.7937708302129786
FGFR1: 0.7588015633893758
MutPred --COL2A1 TP: 251, TN: 168, FP: 13, FN: 3
MutPred --COL4A3 TP: 91, TN: 123, FP: 6, FN: 1
MutPred --COL4A5 TP: 358, TN: 98, FP: 1, FN: 1
MutPred --MYO7A TP: 144, TN: 331, FP: 50, FN: 9
MutPred --FGFR1 TP: 88, TN: 65, FP: 10, FN: 3
MCCs of MutPred predictions for each gene:
COL2A1: 0.9247513759727938
COL4A3: 0.9363283208111267
COL4A5: 0.9871134745786556
MYO7A: 0.7614164739812336
FGFR1: 0.8437104813341852


C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3609092475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3609092475.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df['pathogenicity'] = gene_df['pathogenicity'].astype(int)
C:\Users\InbarBlech\AppData\Local\Temp\ipykernel_26016\3609092475.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score, log_loss, confusion_matrix
# store the results in a dictionary
results = {gene: 0 for gene in genes}

for gene in genes:
    gene_df = merged[merged["gene"] == gene]
    y_true = gene_df["pathogenicity"].astype(int)
    predicted_probabilities = gene_df["MutPred_score"]
    gene_df.loc[:, 'binary_prediction_mutpred'] = (gene_df['MutPred_score'] > 0.5).astype(int)
    predicted_labels = gene_df["binary_prediction_mutpred"]
    
    results_gene = {}
    # Calculate AUC-ROC
    roc_auc = roc_auc_score(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["roc_auc"] = roc_auc
    
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, predicted_probabilities)
    roc_auc = auc(fpr, tpr)
    # append to dictionary
    results_gene["auc"] = roc_auc
    
    # Calculate AUC-PR
    precision, recall, _ = precision_recall_curve(y_true, predicted_probabilities)
    pr_auc = auc(recall, precision)
    # append to dictionary
    results_gene["pr_auc"] = pr_auc
    
    # Calculate F1 Score
    f1 = f1_score(y_true, predicted_labels)
    # append to dictionary
    results_gene["f1"] = f1
    
    # Calculate Log Loss
    logloss = log_loss(y_true, predicted_probabilities)
    # append to dictionary
    results_gene["logloss"] = logloss
    
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(y_true, predicted_labels)
    # append to dictionary
    results_gene["confusion_matrix"] = conf_matrix
    
    # Get gene name for the use for the dictionary
    gene = gene_df['gene'].unique()[0]
    # Append results to dictionary
    results[gene] = results_gene
    
## Print results
print("Results of MutPred2 predictions for each gene:")
for gene in results:
    print(f"{gene}: {results[gene]}")

In [ ]:
# present the confusion matrix for each gene
import matplotlib.pyplot as plt

for gene in results:
    # Plot confusion matrix
    plt.imshow(results[gene]["confusion_matrix"], cmap=plt.cm.Blues)
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.xticks([0, 1], ["Benign", "Pathogenic"])
    plt.yticks([0, 1], ["Benign", "Pathogenic"])
    plt.title(f"Confusion matrix for {gene}, MutPred2 predictions")
    plt.style.use("seaborn-white")
    # Add text annotations
    for i in range(2):
        for j in range(2):
            plt.text(j, i, results[gene]["confusion_matrix"][i, j], ha="center", va="center", color="black", backgroundcolor="white")
    plt.colorbar()
    plt.show()

In [ ]:
# present the ROC curve for each gene
for gene in results:
    # Plot ROC curve
    plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {results[gene]['roc_auc']:.2f})")
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC curve for {gene}, MutPred2 predictions")
    plt.legend(loc="lower right")
    plt.show()